In [1]:
!pip install nba_api

In [3]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, commonplayerinfo
import pandas as pd

In [27]:
def get_player_id(player_name: str) -> str:
    for p in players.get_players():
        if p['full_name'].lower() == player_name.lower():
            return p['id']

    raise NameError('Invalid Player Name; Player Name Does Not Exist')
    return None

def get_player_career_start_year(player_id: str) -> int:
    player_info = commonplayerinfo.CommonPlayerInfo(player_id = player_id)
    career_start_year = player_info.get_data_frames()[0]['FROM_YEAR'][0]

    return int(career_start_year)

player_id = get_player_id('stephen curry')
print(get_player_career_start_year(player_id))

2009


# Interactive Player Selection


In [30]:
player_name = input('Enter Name: ')
player_id = get_player_id(player_name)
start_year = get_player_career_start_year(player_id)
all_seasons_data = pd.DataFrame()

for year in range(start_year, 2024): #Change last parameter if you're not in 2024
    season_str = str(year) + '-' + str(year + 1)[-2:]
    gamelog = playergamelog.PlayerGameLog(player_id = player_id, season = season_str)
    print(gamelog.get_data_frames())

Enter Name: stephen curry
[   SEASON_ID  Player_ID     Game_ID     GAME_DATE      MATCHUP WL  MIN  FGM  \
0      22009     201939  0020901229  APR 14, 2010    GSW @ POR  W   48   13   
1      22009     201939  0020901214  APR 13, 2010  GSW vs. UTA  L   41    5   
2      22009     201939  0020901200  APR 11, 2010  GSW vs. OKC  W   35    9   
3      22009     201939  0020901194  APR 10, 2010    GSW @ LAC  L   41   10   
4      22009     201939  0020901163  APR 07, 2010    GSW @ MIN  W   44   12   
..       ...        ...         ...           ...          ... ..  ...  ...   
75     22009     201939  0020900096  NOV 08, 2009    GSW @ SAC  L   31    4   
76     22009     201939  0020900082  NOV 06, 2009  GSW vs. LAC  L   22    1   
77     22009     201939  0020900069  NOV 04, 2009  GSW vs. MEM  W   28    3   
78     22009     201939  0020900030  OCT 30, 2009    GSW @ PHX  L   39    5   
79     22009     201939  0020900015  OCT 28, 2009  GSW vs. HOU  L   36    7   

    FGA  FG_PCT  ...  DR